# Buoyant driven cavity

In [ ]:
from trustutils.jupyter import run

run.introduction("Yannick Gorsse")
run.description("Driven cavity with cosine thermal power and thermal exchange with uniform external temperature")
run.TRUST_parameters()

In [ ]:
from trustutils.jupyter import run
import os
import sys

run.reset()
run.initBuildDirectory()
sys.path.insert(1, f"{run.BUILD_DIRECTORY}")
run.useMEDCoupling()
from canal import build_mesh
from eos_sodium import Tsat

r = 0.02
h = 1
n = 15
nz = 100
flux = 4e5
power = 0

Tin = 800
P = 1e5

dis = [("VDF", "-+")] #, ("PolyMAC_P0", "x")]
for d, _ in dis:
    run.addCaseFromTemplate(
        f"jdd_{d}.data",
        d,
        {
            "dis": d,
            "power": power,
            "flux": flux,
            "r": r,
            "h": h,
            "r_2": r / 2.0,
            "h_2": h / 2.0,
            "nr": n + 1,
            "nz": nz + 1,
            "Tin": Tin,
            "Tsat": Tsat(P),
            "p": P,
        },
    )

# build_mesh(r, h, n, nz, 2, f"{run.BUILD_DIRECTORY}/PolyMAC_P0", "canal.med")

run.printCases()
run.runCases()
run.tablePerf()


## Pas de temps

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph("", nY=2)
for d, _ in dis:
    dt_ev = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/jdd_{d}.dt_ev")
    a.addPlot(0)
    a.add(dt_ev[:, 0], dt_ev[:, 1], label=d, title="Time steps")
    a.addPlot(1)
    a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/jdd_{d}.dt_ev", label=d)
a.label("Time [s]", "Time steps [s]")
a.scale(xscale='log', yscale='log')

## Profils

In [ ]:
vars = ("V", "T", "A", "TAX")
a = plot.Graph("", nY=len(vars))
for j, var in enumerate(vars):
    a.addPlot(j, f"Variable {var}")
    for d, marker in dis:
        for phase, color in enumerate(["red", "black"]):
            compo = 2 * (1 if var == "V" else 0) + phase
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/jdd_{d}_{var}.son", label=f"{d} - phase {phase}", compo=compo, marker=marker, color=color, markersize=15)


## Puissance

In [ ]:
# import pandas as pd
# from math import pi

# a = plot.Graph("Power")

# df = pd.read_csv(f"{run.BUILD_DIRECTORY}/multi/jdd_pbt_Diffusion_chaleur.out", comment="#", delim_whitespace=True)
# for phase, nom_phase in enumerate(["LIQUIDE_EAU", "GAZ_EAU"]):
#     a.add(df["Time"].to_list(), df[f"wall__{nom_phase}"].to_list(), label=nom_phase, marker="-" if phase == 0 else "o")
# a.add(df["Time"].to_list(), [flux * h * 2.0 * pi * r for _ in df["Time"].to_list()], label="Analytique", marker="--")